# ***PRC analysis*** -- obtaining the phase response curves

The main core of the program.
Use this notebook to analyse the current and calculate PRCs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from config import props, filenames
import analysis
import prc_plotting as plot

Automatic import reloads and pop-up window pyplot plottting:

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

## Data reading, cleaning and interpolation

In [ ]:
data_raw = pd.read_pickle(filenames.data+'data.pkl')
data_raw.t = data_raw.t - data_raw.t[0] # making sure time starts at 0
data = analysis.data_cleaning(data_raw.copy())
pert_times = analysis.find_perts(data)
data = analysis.data_interpolation(data, pert_times)
data = analysis.correct_emsi(data)

The cell below can be ignored

In [ ]:
# plt.figure()
# plt.plot(data.t, np.diff(data.I_real, n=2, prepend=0, append=0))

### Current and emsi waveform plot

Both are plotted so we can make sure everything went right.
This plot is also useful to investigate the current waveform directly.

In [ ]:
plot.current_and_emsi(data, data_raw)

## Analysis

### Calculate periods and pert responses

In [ ]:
cycles = analysis.find_cycles(data, pert_times)
cycles.head(5)

In [ ]:
perts = analysis.pert_response(data, cycles, pert_times[:-2])
perts.head(5)

The function below creates a new column in `perts` called `phase_corrected`.
It is phase wrt. emsi minimum.

In [ ]:
perts = analysis.phase_correction_emsi_minimum(data, perts, cycles)

### Plot period and amplitude drift in time

In [ ]:
plot.period_vs_time(cycles, perts)

In [ ]:
plot.amplitude_vs_time(cycles, perts)

### Retrieving an example period

In [ ]:
example_period_index = cycles[~cycles.had_pert].iloc[100].name
example_period = data[(data.t>cycles.loc[example_period_index].start) & (data.t<cycles.loc[example_period_index+1].start)].copy()
example_period.t = example_period.t-example_period.t.iloc[0]
example_period['phase'] = example_period.t / cycles.loc[example_period_index, 'duration']
if input("Save `example_period.csv`? (possible overwrite) [Y/n]") != 'n':
    example_period.loc[:, ('t', 'phase', 'I', 'I_real', 'emsi', 'emsi_corrected')].to_csv(filenames.data+"example_period.csv", index=False)

## Plotting and saving PRCs

In [ ]:
if props.pert_type == 'U':
    exp_summary = f'PRC for {props.osc_type} {props.voltage}{'+' if props.pert_strength>0 else ''}{props.pert_strength}V, {props.pert_dt}s'
else:
    exp_summary = f'PRC for {props.voltage}V, {props.pert_dt}s {'+' if props.pert_strength>0 else '-'}ve light'
exp_summary

### Plot PRCs

In [ ]:
plot.prc_current(perts, exp_summary)

In [ ]:
plot.amplitude_response(perts, exp_summary)

### Save PRCs

In [ ]:
prc = pd.DataFrame({'phase': perts.phase, 'response': perts.response})
prc = prc.sort_values('phase')
if input("Save `prc.csv`? (possible overwrite) [Y/n]") != 'n':
    prc.to_csv(filenames.data+"prc.csv", index=False)

In [ ]:
if input("Save `prc_full.csv`? (possible overwrite) [Y/n]") != 'n':
    perts.sort_values('phase').to_csv(filenames.data+"prc_full.csv", index=False)

In [ ]:
plot.prc(perts, example_period, exp_summary)

In [ ]:
plot.prc_full(perts, exp_summary)

In [ ]:
plot.amp_response_full(perts, exp_summary)